In [1]:
import os
os.chdir("/home/opossum/Projects/Corona")
os.getcwd()

'/home/opossum/Projects/Corona'

In [2]:
import pandas as pd
import numpy as np
import cufflinks as cf
cf.set_config_file(offline=True, theme='solar')

In [124]:
def read_and_clean_data(name):
    link = "./data/raw/time_series_covid19_{}_global_narrow.csv".format(name)
    df = pd.read_csv(link, skiprows=[1], low_memory=False)
    df["Date"] = pd.to_datetime(df["Date"])
    df_final = df.sort_values("Date", ascending=True)
    df_final.columns = [
        "province",
        "country",
        "lat",
        "long",
        "date",
        "value_{}".format(name),
        "iso_code",
        "region_code",
        "subregion_code",
        "int_region-code"
    ]
    return df_final

def merge_dfs(list_of_dfs):
    df_main = list_of_dfs[0].copy()
    for df in list_of_dfs[1:]:
        df_main = pd.merge(df_main, df, how='outer', on=[
            "province",
            "country",
            "lat",
            "long",
            "date",
            "iso_code",
            "region_code",
            "subregion_code",
            "int_region-code",
        ])
    return df_main

def data_connect():
    df_con = read_and_clean_data("confirmed")
    df_dth = read_and_clean_data("deaths")
    df_rec = read_and_clean_data("recovered")

    df_merged = merge_dfs([df_con, df_dth, df_rec])
    
    df_grouped_country = df_merged.groupby(by=["country", "date"], as_index=False)[["value_deaths", "value_recovered", "value_confirmed"]].sum()
    
    df_world = df_grouped_country.groupby("date", as_index=False).sum()
    df_world["country"] = "World"
    
    df_main = df_grouped_country.append(df_world)
    return df_main

In [125]:
df_main = data_connect()

In [127]:
df_main[df_main["country"] == "World"]

,country,date,value_deaths,value_recovered,value_confirmed
0,World,2020-01-22,17.0,28.0,555.0
1,World,2020-01-23,18.0,30.0,654.0
2,World,2020-01-24,26.0,36.0,941.0
3,World,2020-01-25,42.0,39.0,1434.0
4,World,2020-01-26,56.0,52.0,2118.0
...,...,...,...,...,...
247,World,2020-09-25,988864.0,22472106.0,32562075.0
248,World,2020-09-26,994143.0,22715380.0,32840012.0
249,World,2020-09-27,997734.0,22925667.0,33077724.0
250,World,2020-09-28,1001646.0,23151081.0,33353615.0
